In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说,这份code的目标只是为了提升模型的分类效率

In [7]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
import model_tunning as mt
from param_config import config

In [8]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']

###清理异常Y值
#异常大值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]

#性别空
dfTrain = dfTrain.loc[~dfTrain['sex'].isnull()]

dfTrain = dfTrain.reset_index(drop=True)

saveY = dfTrain['Y'].tolist()

dfTrain.loc[dfTrain['Y']>dfTrain['Y'].quantile(0.9),'Y'] = 1
dfTrain.loc[dfTrain['Y']!=1,'Y'] = 0

In [9]:
def FeatAll(train,pred,rankMinus=False,missing=False,missing_step2=False):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days
    
    ###RF填补缺失值
    if missing:
        dfAll = fe.fillna_RF(dfAll,predictors,True)

    
    ###尝试处理异常值
    '''for var in predictors:
        dfAll.loc[dfAll[var]>=dfAll[var].quantile(0.99)*3,var] = dfAll[var].quantile(0.99)*3'''
        #dfAll.loc[dfAll[var]>=df[var].quantile(0.99)*1.5,var] = np.nan
    ###根据年龄性别去做分类，鉴别不同人的百分比
    dfAll['ageBin'] = 0
    ageBin = [20,30,40,50,60]
    for i in range(len(ageBin)):
        dfAll.loc[dfAll['age']>ageBin[i],'ageBin'] = i+1
    dfAll['sexAge'] = dfAll[['sex','ageBin']].apply(lambda x: x[1]*2+x[0])
    dfAllPcentBySexAge = fe.pcent_by_other_col(dfAll,{'sex':predictors,'ageBin':predictors,'sexAge':predictors},['ID'])
    del dfAll['age']
    del dfAll['sexAge']
    if rankMinus:
        try:
            for i in ['_sex_','_ageBin_','_sexAge']:
                dfrankMinusTmp = pd.read_csv('../../Cache/rankMinus%s.csv'%i[:-1])
                if i == '_sex_':
                    dfrankMinus = dfrankMinusTmp.copy()
                else:
                    dfrankMinus = dfrankMinus.merge(dfrankMinusTmp,'inner','ID')
        except:
            for i in ['_sex_','_ageBin_','_sexAge']:
                varList = [var for var in dfAllPcentBySexAge.columns if i in var]
                dfrankMinusTmp = fe.var_minus(dfAllPcentBySexAge,varList,['ID'],0)
                dfrankMinusTmp.to_csv('../../Cache/rankMinus%s.csv'%i[:-1],index=False)
                if i == '_sex_':
                    dfrankMinus = dfrankMinusTmp.copy()
                else:
                    dfrankMinus = dfrankMinus.merge(dfrankMinusTmp,'inner','ID')  

    
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in
    
    for plus in [['PartII_1','PartII_2'],['PartIII_1','PartIII_2','PartIII_3']]:
        tmpVar =plus[0]
        tmpValue = dfAll[plus[0]].values
        for var in plus[1:]:
            tmpVar = tmpVar +'_plus_' + var
            tmpValue = tmpValue + dfAll[var].values
        dfAll[tmpVar] = tmpValue
    
    dfAll['PartI_9'] = dfAll['PartI_5']-dfAll['PartI_6']-dfAll['PartI_7']
    dfAll['PartII_5'] = dfAll['PartII_2']-dfAll['PartII_3']-dfAll['PartII_4']
    
    for ratio in [['PartII_1','PartII_2'],['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]
        
    for multiply in [['PartI_1','PartI_5'],['PartI_2','PartI_5'],['PartI_3','PartI_5'],['PartI_4','PartI_5'],['PartVI_1','PartVI_2'],['PartV_2','PartV_5'],['PartV_2','PartV_7'],['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]*dfAll[multiply[1]]
    
    
    
    try:
        dfAll = dfAll.merge(dfAllPcentBySexAge,'inner','ID')
    except:
        print('No Pcent by sex and age')
        
    try:
        dfAll = dfAll.merge(dfrankMinus,'inner','ID')
    except:
        print('No Rank Minus')    

   
    
    
    ###填补缺失值
    if missing_step2:
        dfAll =dfAll.fillna(dfAll.median())
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred
    

In [4]:
dfTrain,dfPred = FeatAll(dfTrain,dfPred,True,True,True)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


No Rank Minus


In [5]:
len(predictors)

172

In [6]:
n_splits=5
early_stop=50
z,x,c = mm.catboost_kfold(dfTrain,dfPred,predictors,n_splits,early_stop = early_stop)

it is a classifier problem


C:\Users\Leo Mao\Anaconda3\lib\site-packages\catboost\core.py:1252: FutureWarning: The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (posible values: 'Silent', 'Verbose', 'Info', 'Debug').
  super(CatBoostClassifier, self).__init__(params)


0:	learn: 0.6836156	test: 0.6835876	best: 0.6835876 (0)	total: 103ms	remaining: 3m 26s
1:	learn: 0.6740454	test: 0.6740005	best: 0.6740005 (1)	total: 130ms	remaining: 2m 10s
2:	learn: 0.6652861	test: 0.6652659	best: 0.6652659 (2)	total: 149ms	remaining: 1m 39s
3:	learn: 0.6549931	test: 0.6549666	best: 0.6549666 (3)	total: 174ms	remaining: 1m 26s
4:	learn: 0.6470002	test: 0.6469730	best: 0.6469730 (4)	total: 194ms	remaining: 1m 17s
5:	learn: 0.6382078	test: 0.6382061	best: 0.6382061 (5)	total: 217ms	remaining: 1m 12s
6:	learn: 0.6283514	test: 0.6282532	best: 0.6282532 (6)	total: 236ms	remaining: 1m 7s
7:	learn: 0.6209363	test: 0.6208745	best: 0.6208745 (7)	total: 259ms	remaining: 1m 4s
8:	learn: 0.6135028	test: 0.6135197	best: 0.6135197 (8)	total: 279ms	remaining: 1m 1s
9:	learn: 0.6050973	test: 0.6051395	best: 0.6051395 (9)	total: 301ms	remaining: 60s
10:	learn: 0.5981026	test: 0.5981727	best: 0.5981727 (10)	total: 323ms	remaining: 58.4s
11:	learn: 0.5904241	test: 0.5904930	best: 0.590

98:	learn: 0.3189109	test: 0.3197335	best: 0.3197335 (98)	total: 2.12s	remaining: 40.7s
99:	learn: 0.3180829	test: 0.3189297	best: 0.3189297 (99)	total: 2.14s	remaining: 40.6s
100:	learn: 0.3171292	test: 0.3179699	best: 0.3179699 (100)	total: 2.16s	remaining: 40.6s
101:	learn: 0.3158590	test: 0.3166990	best: 0.3166990 (101)	total: 2.18s	remaining: 40.6s
102:	learn: 0.3149326	test: 0.3158178	best: 0.3158178 (102)	total: 2.2s	remaining: 40.5s
103:	learn: 0.3137764	test: 0.3146420	best: 0.3146420 (103)	total: 2.22s	remaining: 40.4s
104:	learn: 0.3129640	test: 0.3138015	best: 0.3138015 (104)	total: 2.24s	remaining: 40.4s
105:	learn: 0.3121315	test: 0.3131113	best: 0.3131113 (105)	total: 2.26s	remaining: 40.3s
106:	learn: 0.3115091	test: 0.3125264	best: 0.3125264 (106)	total: 2.28s	remaining: 40.3s
107:	learn: 0.3104060	test: 0.3114684	best: 0.3114684 (107)	total: 2.3s	remaining: 40.3s
108:	learn: 0.3095450	test: 0.3106974	best: 0.3106974 (108)	total: 2.32s	remaining: 40.3s
109:	learn: 0.30

196:	learn: 0.2658033	test: 0.2714418	best: 0.2714418 (196)	total: 4.13s	remaining: 37.8s
197:	learn: 0.2655009	test: 0.2711686	best: 0.2711686 (197)	total: 4.15s	remaining: 37.8s
198:	learn: 0.2651605	test: 0.2709426	best: 0.2709426 (198)	total: 4.17s	remaining: 37.7s
199:	learn: 0.2648603	test: 0.2706566	best: 0.2706566 (199)	total: 4.19s	remaining: 37.7s
200:	learn: 0.2645456	test: 0.2703724	best: 0.2703724 (200)	total: 4.21s	remaining: 37.7s
201:	learn: 0.2642529	test: 0.2701335	best: 0.2701335 (201)	total: 4.23s	remaining: 37.7s
202:	learn: 0.2638984	test: 0.2697907	best: 0.2697907 (202)	total: 4.25s	remaining: 37.6s
203:	learn: 0.2636389	test: 0.2695987	best: 0.2695987 (203)	total: 4.28s	remaining: 37.6s
204:	learn: 0.2633807	test: 0.2693924	best: 0.2693924 (204)	total: 4.29s	remaining: 37.6s
205:	learn: 0.2631936	test: 0.2692901	best: 0.2692901 (205)	total: 4.32s	remaining: 37.6s
206:	learn: 0.2629331	test: 0.2690827	best: 0.2690827 (206)	total: 4.33s	remaining: 37.5s
207:	learn

...

977:	learn: 0.1906746	test: 0.2389546	best: 0.2388969 (971)	total: 20.3s	remaining: 21.2s
978:	learn: 0.1905680	test: 0.2389557	best: 0.2388969 (971)	total: 20.3s	remaining: 21.2s
979:	learn: 0.1904973	test: 0.2389463	best: 0.2388969 (971)	total: 20.3s	remaining: 21.2s
980:	learn: 0.1904424	test: 0.2389605	best: 0.2388969 (971)	total: 20.4s	remaining: 21.1s
981:	learn: 0.1903783	test: 0.2389633	best: 0.2388969 (971)	total: 20.4s	remaining: 21.1s
982:	learn: 0.1903170	test: 0.2389926	best: 0.2388969 (971)	total: 20.4s	remaining: 21.1s
983:	learn: 0.1902918	test: 0.2389992	best: 0.2388969 (971)	total: 20.4s	remaining: 21.1s
984:	learn: 0.1901956	test: 0.2390198	best: 0.2388969 (971)	total: 20.4s	remaining: 21.1s
985:	learn: 0.1901095	test: 0.2390073	best: 0.2388969 (971)	total: 20.4s	remaining: 21s
986:	learn: 0.1900777	test: 0.2390109	best: 0.2388969 (971)	total: 20.5s	remaining: 21s
987:	learn: 0.1900290	test: 0.2389927	best: 0.2388969 (971)	total: 20.5s	remaining: 21s
988:	learn: 0.18

1078:	learn: 0.1845384	test: 0.2381512	best: 0.2381512 (1078)	total: 22.3s	remaining: 19s
1079:	learn: 0.1845161	test: 0.2381405	best: 0.2381405 (1079)	total: 22.3s	remaining: 19s
1080:	learn: 0.1844590	test: 0.2381592	best: 0.2381405 (1079)	total: 22.3s	remaining: 19s
1081:	learn: 0.1844150	test: 0.2381789	best: 0.2381405 (1079)	total: 22.4s	remaining: 19s
1082:	learn: 0.1843847	test: 0.2381648	best: 0.2381405 (1079)	total: 22.4s	remaining: 18.9s
1083:	learn: 0.1843386	test: 0.2381576	best: 0.2381405 (1079)	total: 22.4s	remaining: 18.9s
1084:	learn: 0.1842535	test: 0.2381318	best: 0.2381318 (1084)	total: 22.4s	remaining: 18.9s
1085:	learn: 0.1841974	test: 0.2381562	best: 0.2381318 (1084)	total: 22.4s	remaining: 18.9s
1086:	learn: 0.1841410	test: 0.2381321	best: 0.2381318 (1084)	total: 22.5s	remaining: 18.9s
1087:	learn: 0.1841141	test: 0.2381221	best: 0.2381221 (1087)	total: 22.5s	remaining: 18.8s
1088:	learn: 0.1840247	test: 0.2381312	best: 0.2381221 (1087)	total: 22.5s	remaining: 18

1177:	learn: 0.1791965	test: 0.2374607	best: 0.2374178 (1175)	total: 24.3s	remaining: 16.9s
1178:	learn: 0.1791510	test: 0.2374638	best: 0.2374178 (1175)	total: 24.3s	remaining: 16.9s
1179:	learn: 0.1791156	test: 0.2374615	best: 0.2374178 (1175)	total: 24.3s	remaining: 16.9s
1180:	learn: 0.1790534	test: 0.2374643	best: 0.2374178 (1175)	total: 24.3s	remaining: 16.9s
1181:	learn: 0.1790098	test: 0.2374286	best: 0.2374178 (1175)	total: 24.4s	remaining: 16.9s
1182:	learn: 0.1789426	test: 0.2374236	best: 0.2374178 (1175)	total: 24.4s	remaining: 16.8s
1183:	learn: 0.1789212	test: 0.2374359	best: 0.2374178 (1175)	total: 24.4s	remaining: 16.8s
1184:	learn: 0.1788675	test: 0.2374679	best: 0.2374178 (1175)	total: 24.4s	remaining: 16.8s
1185:	learn: 0.1788094	test: 0.2374497	best: 0.2374178 (1175)	total: 24.4s	remaining: 16.8s
1186:	learn: 0.1787536	test: 0.2374461	best: 0.2374178 (1175)	total: 24.5s	remaining: 16.7s
1187:	learn: 0.1787036	test: 0.2373967	best: 0.2373967 (1187)	total: 24.5s	remai

1275:	learn: 0.1741763	test: 0.2369925	best: 0.2369704 (1240)	total: 26.2s	remaining: 14.9s
1276:	learn: 0.1741145	test: 0.2369843	best: 0.2369704 (1240)	total: 26.3s	remaining: 14.9s
1277:	learn: 0.1740336	test: 0.2368933	best: 0.2368933 (1277)	total: 26.3s	remaining: 14.8s
1278:	learn: 0.1739713	test: 0.2368529	best: 0.2368529 (1278)	total: 26.3s	remaining: 14.8s
1279:	learn: 0.1739178	test: 0.2368388	best: 0.2368388 (1279)	total: 26.3s	remaining: 14.8s
1280:	learn: 0.1738728	test: 0.2368086	best: 0.2368086 (1280)	total: 26.3s	remaining: 14.8s
1281:	learn: 0.1738063	test: 0.2368003	best: 0.2368003 (1281)	total: 26.4s	remaining: 14.8s
1282:	learn: 0.1737155	test: 0.2368157	best: 0.2368003 (1281)	total: 26.4s	remaining: 14.7s
1283:	learn: 0.1736677	test: 0.2368448	best: 0.2368003 (1281)	total: 26.4s	remaining: 14.7s
1284:	learn: 0.1736357	test: 0.2368389	best: 0.2368003 (1281)	total: 26.4s	remaining: 14.7s
1285:	learn: 0.1735637	test: 0.2367785	best: 0.2367785 (1285)	total: 26.4s	remai

1374:	learn: 0.1693405	test: 0.2365021	best: 0.2363445 (1342)	total: 28.2s	remaining: 12.8s
1375:	learn: 0.1692797	test: 0.2364561	best: 0.2363445 (1342)	total: 28.2s	remaining: 12.8s
1376:	learn: 0.1692214	test: 0.2364659	best: 0.2363445 (1342)	total: 28.2s	remaining: 12.8s
1377:	learn: 0.1691649	test: 0.2364751	best: 0.2363445 (1342)	total: 28.3s	remaining: 12.8s
1378:	learn: 0.1691366	test: 0.2364708	best: 0.2363445 (1342)	total: 28.3s	remaining: 12.7s
1379:	learn: 0.1690699	test: 0.2364883	best: 0.2363445 (1342)	total: 28.3s	remaining: 12.7s
1380:	learn: 0.1690201	test: 0.2364409	best: 0.2363445 (1342)	total: 28.3s	remaining: 12.7s
1381:	learn: 0.1689851	test: 0.2364341	best: 0.2363445 (1342)	total: 28.3s	remaining: 12.7s
1382:	learn: 0.1689422	test: 0.2364355	best: 0.2363445 (1342)	total: 28.4s	remaining: 12.7s
1383:	learn: 0.1688876	test: 0.2364827	best: 0.2363445 (1342)	total: 28.4s	remaining: 12.6s
1384:	learn: 0.1688557	test: 0.2364698	best: 0.2363445 (1342)	total: 28.4s	remai

In [ ]:
predictorsRF = fs.RF_selection(dfTrain,predictors)

In [ ]:
c.tree_count_

## 分类模型

In [ ]:
n_splits=5
early_stop=50
modelPredictors = predictorsRF[:500]
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'binary:logistic','lambda':1,'subsample':0.8,'colsample_bytree':0.8,'eval_metric':'logloss'}
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,modelPredictors,n_splits,early_stop = early_stop,params=params)

In [ ]:
mt.evaluate_performance(test_result['target'],test_result['score'])

In [ ]:
selectNum = 150
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[:selectNum]
imp.sort_values('sum_imp',ascending=False)

In [ ]:
test_result,result,imp_selected = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop = early_stop,params=params)

In [ ]:
mt.evaluate_performance(test_result['target'],test_result['score'])
imp.sort_values('sum_imp',ascending=False)

## 回归模型

In [ ]:
##将分类打分放入原始数据
dfTrain['scoreBinary'] = test_result['score']
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
dfPred['scoreBinary'] = result['score'].tolist()

predictorReg = predictors.copy()
predictorReg.append('scoreBinary')

dfTrain['Y'] = saveY

##将分数以rank形式保存一份
'''dfTrain['scoreBinaryPcent'] = dfTrain['scoreBinary'].rank()/float(dfTrain.shape[0])
dfPred['scoreBinaryPcent'] = dfPred['scoreBinary'].rank()/float(dfPred.shape[0])
predictorReg.append('scoreBinaryPcent')'''

In [ ]:
print(dfTrain['scoreBinary'].mean())
print(dfTrain['scoreBinary'].max())
print(dfTrain['scoreBinary'].min())
print('--------------')
print(dfPred['scoreBinary'].mean())
print(dfPred['scoreBinary'].max())
print(dfPred['scoreBinary'].min())

In [ ]:
predictorsRF = fs.RF_selection(dfTrain,predictorReg)
modelPredictors = predictorsRF[:500]

In [ ]:
n_splits=5
early_stop=50
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,'colsample_bytree':0.8}
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,modelPredictors,n_splits,early_stop=early_stop,params=params)

In [ ]:
selectNum = 100
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:selectNum+1]
imp.sort_values('sum_imp',ascending=False)

In [ ]:
test_result,result,imp_selected = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop=early_stop,params=params)

In [ ]:
for i in range(1,n_splits+1):
    imp_selected['imp_fold%d'%i] = imp_selected['imp_fold%d'%i]/imp_selected['imp_fold%d'%i].sum()
imp_selected['sum_imp'] = imp_selected[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)
imp_selected.sort_values('sum_imp',ascending=False).reset_index()

In [ ]:
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
print((result['score']>6.8).sum())
print(result['score'].min())
print(result['score'].max())
print(result['score'].mean())
print(result['score'].median())
print("-------------------------")
print(result.iloc[938,:])

In [ ]:
n_splits=5
other_note ='_classfier_first'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)